In [8]:
import numpy as np 
import glob
import os
import matplotlib.pyplot as plt
import string
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from sklearn.svm import SVC
# HOG

import json
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from subprocess import check_output

dim = 100

In [15]:
!pip3 install opencv-python

     |████████████████████████████████| 42.6 MB 1.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [16]:
def GetFruit(fruitlist,datatype,print_n=False,k_fold=False):
    images=[]
    labels=[]
    val=['train','test']
    if not k_fold:
        PATH="./fruits-360/"+datatype+"/"
        for i,fruit in enumerate(fruitlist):
            p=PATH+fruit
            j=0
            for image_path in glob.glob(os.path.join(p,"*.jpg")):
                image=cv2.imread(image_path,cv2.IMREAD_COLOR)
                image=cv2.resize(image,(dim,dim))
                image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
                images.append(image)
                labels.append(i)
                j+=1
            if(print_n):
                print("There are",j,"",datatype.upper()," images of ",fruitlist[i].upper())
        images=np.array(images)
        labels=np.array(labels)
        return images,labels
    else:
        for v in val:
            PATH="./fruits-360/"+v+"/"
            for i,fruit in enumerate(fruitlist):
                p=PATH+fruit
                j=0
                for image_path in glob.glob(os.path.join(p,"*.jpg")):
                    image=cv2.imread(image_path,cv2.IMREAD_COLOR)
                    image=cv2.resize(image,(dim,dim))
                    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
                    images.append(image)
                    labels.append(i)
                    j+=1
            images=np.array(images)
            labels=np.array(labels)
            return images,labels


In [ ]:
# simple Binary-classification problem
# Pineapple and kiwi

In [19]:
import cv2
fruitlist=['Pineapple','kiwi']
# Get image and Labels
X_train_raw,y_train=GetFruit(fruitlist,'train',print_n=True,k_fold=False)
X_test_raw,y_test=GetFruit(fruitlist,'test',print_n=True,k_fold=False)
# Get data for k-fold
X_raw,y=GetFruit(fruitlist,'',print_n=True,k_fold=True)

There are 490  TRAIN  images of  PINEAPPLE
There are 466  TRAIN  images of  KIWI
There are 166  TEST  images of  PINEAPPLE
There are 156  TEST  images of  KIWI


In [20]:
from skimage import feature
ppc=16
hog_features_train = []
hog_features_test = []
hog_images_train= []
hog_images_test=[]
for image in X_train_raw:
    fd,hog_image = feature.hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images_train.append(hog_image)
    hog_features_train.append(fd)
for image in X_test_raw:
    fd,hog_image = feature.hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images_test.append(hog_image)
    hog_features_test.append(fd)

# Scale data Images
scaler=StandardScaler()
X_train=scaler.fit_transform([i.flatten() for i in X_train_raw])
X_test=scaler.fit_transform([i.flatten() for i in X_test_raw])
X_train_hog=scaler.fit_transform([i.flatten() for i in hog_images_train])
X_test_hog=scaler.fit_transform([i.flatten() for i in hog_images_test])
X=scaler.fit_transform([i.flatten() for i in X_raw])

In [21]:
print("Shape of the data:")
print((X_train.shape,y_train.shape))
print((X_test.shape,y_test.shape))
print(X_train_hog.shape)
print(X_test_hog.shape)

print("\nData sample:")
print((X_train[0],y_train[0]))
print((X_train_hog[0],y_train[0]))

Shape of the data:
((956, 30000), (956,))
((322, 30000), (322,))
(956, 10000)
(322, 10000)

Data sample:
(array([0.23243119, 0.24001185, 0.26975539, ..., 0.        , 0.        ,
       0.        ]), 0)
(array([0., 0., 0., ..., 0., 0., 0.]), 0)
